In [ ]:
!pip install llama-index

# 检索器

在LlamaIndex中，检索器负责根据用户查询获取最相关的上下文。检索器建立在索引之上，指定如何从索引中提取节点。我们在前一课中已经讨论了索引。

LlamaIndex支持许多类型的检索器，包括以下内容：

**向量存储检索器**

向量存储检索器从向量存储索引中提取前k个最相似的节点。这里模式没有重要性。

**列表检索器**

您可以使用列表检索器从列表索引中提取所有节点。此检索器支持两种模式：默认模式和嵌入模式。

默认模式提取所有节点，而嵌入模式使用嵌入提取前k个节点。

**树检索器**

正如其名称所示，树检索器从节点的分层树中提取节点。此检索器支持许多不同的模式，默认模式是select_leaf。

**关键词表检索器**

关键词表检索器从查询中提取关键词，并使用这些关键词查找具有匹配关键词的节点。

该检索器支持三种不同的模式：默认、简单和rake。

**知识图检索器**

知识图检索器从节点的分层树中提取节点。

它支持关键词、嵌入和混合模式。混合模式同时使用关键词和嵌入查找相关三元组。

这里是一个基于列表索引构建的检索器的代码示例

In [6]:
from llama_index import ListIndex
from llama_index import download_loader

YoutubeTranscriptReader = download_loader("YoutubeTranscriptReader")

loader = YoutubeTranscriptReader()
docs = loader.load_data(ytlinks=['https://www.youtube.com/watch?v=nHcbHdgVUJg&ab_channel=WintWealth'])
list_index = ListIndex(docs)
retriever = list_index.as_retriever(
    retriever_mode='embedding',
)

# 节点后处理器

LlamaIndex中的节点后处理器接受一组节点，并在返回它们之前应用转换或过滤。

节点后处理器通常在查询引擎中使用，在检索步骤之后和响应合成步骤之前。

例如，您可以要求检索到的节点中不存在或存在特定关键字。您还可以根据时间等属性对结果进行排名。

有许多类型的后处理器，让我们讨论其中一些：

**SimilarityPostprocessor**

允许您要求检索到的节点具有最小相似度分数。

**KeywordNodePostprocessor**

允许您要求节点中存在特定关键字。

**TimeWeightedPostprocessor**

TimeWeightedPostprocessor根据最近性对节点进行排名。

**PIINodePostprocessor**

PIINodeprocessor使用诸如StableLM之类的本地LLM对文本中的个人身份信息（PII）进行屏蔽。例如，将信用卡号替换为[CREDIT_CARD_NUMBER]。

**FixedRecencyPostprocessor**

如果查询涉及时间因素（时间查询），此后处理器将按日期对节点进行排序，并将前k个节点返回给响应合成器。

让我们看一个SimilarityPostprocessor的代码示例

In [9]:
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.schema import Node, NodeWithScore

nodes = [
  NodeWithScore(node=Node(text="text"), score=0.7),
  NodeWithScore(node=Node(text="text"), score=0.8)
]

# filter nodes below 0.75 similarity score
processor = SimilarityPostprocessor(similarity_cutoff=0.75)
filtered_nodes = processor.postprocess_nodes(nodes)

In [10]:
filtered_nodes

[NodeWithScore(node=TextNode(id_='45c4ba0f-edac-4b2b-bcaa-ae9dd3d5210a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='8a363c3e689f3fd0f4d742d70375383ef3906cc7a90e6f9ee402405a72de759c', text='text', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8)]

如您所见，得分为0.7的节点被过滤掉了

# 响应合成器

响应合成器接受节点列表作为输入，并生成响应形式的输出。

LlamaIndex支持不同的响应合成模式，包括：

**Refine**

一种迭代生成响应的方式，使用所有节点。初始答案是使用第一个节点作为上下文生成的。然后将此答案和第二个节点作为上下文提供给一个改进提示。这个过程一直持续，直到所有节点都以相同方式处理，并生成最终响应。

**Compact and Refine**

与Refine相同，只是将文本块合并为更大的块，以优化成本和性能。

**Tree Summarize**

树摘要是一种自底向上的方法，从节点列表构建响应。它涉及通过组合每两个节点生成摘要和父节点，直到只剩下一个答案。

**Simple Summarize**

将所有文本块合并，并使用整个文本作为上下文进行一次LLM调用。

In [ ]:
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

# build index
index = VectorStoreIndex.from_documents(docs)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("What did the author do growing up?")
print(response)
